In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import acf, adfuller, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly.express as px
from datetime import datetime
import matplotlib.pyplot as plt
import io
import base64
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN Y CARGA DE DATOS
# ============================================================================

def load_data(filepath):
    """Carga el dataset de criptomonedas"""
    df = pd.read_csv(filepath)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Symbol', 'Date'])
    return df

# ============================================================================
# ANÁLISIS DE SERIES TEMPORALES
# ============================================================================

def calculate_returns(prices):
    """Calcula los retornos logarítmicos"""
    return np.log(prices / prices.shift(1))

def adf_test(series, name="Series"):
    """Realiza el test Augmented Dickey-Fuller"""
    result = adfuller(series.dropna())
    output = {
        'Test Statistic': result[0],
        'p-value': result[1],
        'Critical Values': result[4],
        'Is Stationary': result[1] < 0.05
    }
    return output

def calculate_acf_values(series, nlags=40):
    """Calcula valores de ACF"""
    series_clean = series.dropna()
    acf_values = acf(series_clean, nlags=nlags)
    return acf_values

def calculate_pacf_values(series, nlags=40):
    """Calcula valores de PACF"""
    series_clean = series.dropna()
    try:
        pacf_values = pacf(series_clean, nlags=nlags)
    except:
        pacf_values = np.zeros(nlags + 1)
    return pacf_values

# ============================================================================
# VISUALIZACIONES MEJORADAS CON ACF PLOTS ESTILO STATSMODELS
# ============================================================================

def plot_price_and_returns(df_crypto, crypto_name):
    """Genera gráfica de precios y retornos"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f'{crypto_name} - Price Evolution',
                       f'{crypto_name} - Daily Returns'),
        vertical_spacing=0.12,
        row_heights=[0.5, 0.5]
    )

    # Precios
    fig.add_trace(
        go.Scatter(x=df_crypto['Date'], y=df_crypto['Close'],
                  name='Close Price', line=dict(color='#1f77b4', width=2)),
        row=1, col=1
    )

    # Retornos
    fig.add_trace(
        go.Scatter(x=df_crypto['Date'], y=df_crypto['Returns'],
                  name='Returns', line=dict(color='#ff7f0e', width=1)),
        row=2, col=1
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Returns", row=2, col=1)

    fig.update_layout(
        height=700,
        showlegend=True,
        title_text=f"{crypto_name} - Price and Returns Analysis",
        hovermode='x unified'
    )

    return fig

def create_acf_plot_improved(series, title, nlags=40, color='#0066cc'):
    """Crea un ACF plot mejorado estilo statsmodels con interpretación visual"""

    series_clean = series.dropna()
    acf_values = acf(series_clean, nlags=nlags)

    # Calcular intervalo de confianza
    conf_interval = 1.96 / np.sqrt(len(series_clean))

    fig = go.Figure()

    # Añadir líneas verticales desde el eje x hasta cada valor ACF
    for i in range(len(acf_values)):
        fig.add_trace(go.Scatter(
            x=[i, i],
            y=[0, acf_values[i]],
            mode='lines',
            line=dict(color=color, width=2),
            showlegend=False,
            hovertemplate=f'Lag {i}<br>ACF: {acf_values[i]:.4f}<extra></extra>'
        ))

    # Añadir puntos en la punta de cada línea
    fig.add_trace(go.Scatter(
        x=list(range(len(acf_values))),
        y=acf_values,
        mode='markers',
        marker=dict(size=6, color=color, symbol='circle'),
        showlegend=False,
        hovertemplate='Lag %{x}<br>ACF: %{y:.4f}<extra></extra>'
    ))

    # Añadir bandas de confianza
    fig.add_hline(y=conf_interval, line_dash="dash", line_color="red",
                  annotation_text="95% Confidence", annotation_position="right")
    fig.add_hline(y=-conf_interval, line_dash="dash", line_color="red")

    # Añadir área de confianza sombreada
    fig.add_hrect(y0=-conf_interval, y1=conf_interval,
                  fillcolor="rgba(255, 0, 0, 0.1)",
                  layer="below", line_width=0)

    fig.update_layout(
        title=title,
        xaxis_title="Lag",
        yaxis_title="Autocorrelation",
        height=400,
        plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgray',
            zeroline=False
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgray',
            zeroline=False,
            range=[-0.1, 1.1]
        )
    )

    return fig

def create_pacf_plot_improved(series, title, nlags=40, color='#ff6600'):
    """Crea un PACF plot mejorado estilo statsmodels"""

    series_clean = series.dropna()
    try:
        pacf_values = pacf(series_clean, nlags=nlags)
    except:
        pacf_values = np.zeros(nlags + 1)

    # Calcular intervalo de confianza
    conf_interval = 1.96 / np.sqrt(len(series_clean))

    fig = go.Figure()

    # Añadir líneas verticales desde el eje x hasta cada valor PACF
    for i in range(len(pacf_values)):
        fig.add_trace(go.Scatter(
            x=[i, i],
            y=[0, pacf_values[i]],
            mode='lines',
            line=dict(color=color, width=2),
            showlegend=False,
            hovertemplate=f'Lag {i}<br>PACF: {pacf_values[i]:.4f}<extra></extra>'
        ))

    # Añadir puntos en la punta de cada línea
    fig.add_trace(go.Scatter(
        x=list(range(len(pacf_values))),
        y=pacf_values,
        mode='markers',
        marker=dict(size=6, color=color, symbol='circle'),
        showlegend=False,
        hovertemplate='Lag %{x}<br>PACF: %{y:.4f}<extra></extra>'
    ))

    # Añadir bandas de confianza
    fig.add_hline(y=conf_interval, line_dash="dash", line_color="red",
                  annotation_text="95% Confidence", annotation_position="right")
    fig.add_hline(y=-conf_interval, line_dash="dash", line_color="red")
    fig.add_hline(y=0, line_color="black", line_width=1)

    # Añadir área de confianza sombreada
    fig.add_hrect(y0=-conf_interval, y1=conf_interval,
                  fillcolor="rgba(255, 0, 0, 0.1)",
                  layer="below", line_width=0)

    fig.update_layout(
        title=title,
        xaxis_title="Lag",
        yaxis_title="Partial Autocorrelation",
        height=400,
        plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgray',
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=1
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgray',
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=1,
            range=[-0.5, 1.1]
        )
    )

    return fig

def plot_acf_analysis_complete(df_crypto, crypto_name, nlags=40):
    """Genera análisis completo de ACF para precios, retornos y retornos²"""

    # Crear 3 gráficas: ACF para precios, retornos y retornos²
    plots = []

    # 1. ACF de Precios
    plots.append(create_acf_plot_improved(
        df_crypto['Close'],
        f'{crypto_name} - ACF of Prices (Evidence of Random Walk)',
        nlags=nlags,
        color='#0066cc'
    ))

    # 2. ACF de Retornos
    plots.append(create_acf_plot_improved(
        df_crypto['Returns'],
        f'{crypto_name} - ACF of Returns (Test for Independence)',
        nlags=nlags,
        color='#ff6600'
    ))

    # 3. ACF de Retornos² (Volatilidad)
    plots.append(create_acf_plot_improved(
        df_crypto['Returns']**2,
        f'{crypto_name} - ACF of Squared Returns (Volatility Clustering)',
        nlags=nlags,
        color='#00aa00'
    ))

    return plots

def plot_volatility_clustering(df_crypto, crypto_name, window=30):
    """Visualiza el clustering de volatilidad"""

    # Calcular volatilidad móvil
    df_crypto['Rolling_Vol'] = df_crypto['Returns'].rolling(window=window).std()

    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            f'{crypto_name} - Returns with Volatility Periods',
            f'{crypto_name} - Rolling Volatility ({window}-day window)'
        ),
        vertical_spacing=0.12
    )

    # Retornos coloreados por volatilidad
    fig.add_trace(
        go.Scatter(
            x=df_crypto['Date'],
            y=df_crypto['Returns'],
            mode='lines',
            name='Returns',
            line=dict(color='#ff7f0e', width=1),
            fill='tozeroy',
            fillcolor='rgba(255, 127, 14, 0.2)'
        ),
        row=1, col=1
    )

    # Volatilidad móvil
    fig.add_trace(
        go.Scatter(
            x=df_crypto['Date'],
            y=df_crypto['Rolling_Vol'],
            name='Rolling Volatility',
            line=dict(color='#d62728', width=2)
        ),
        row=2, col=1
    )

    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Returns", row=1, col=1)
    fig.update_yaxes(title_text="Volatility (Std Dev)", row=2, col=1)

    fig.update_layout(
        height=700,
        title_text=f"{crypto_name} - Volatility Clustering Analysis",
        hovermode='x unified'
    )

    return fig

def plot_returns_distribution(df_crypto, crypto_name):
    """Distribución de retornos"""
    fig = go.Figure()

    fig.add_trace(go.Histogram(
        x=df_crypto['Returns'].dropna(),
        nbinsx=50,
        name='Returns Distribution',
        marker_color='#9467bd',
        opacity=0.7
    ))

    fig.update_layout(
        title=f"{crypto_name} - Returns Distribution",
        xaxis_title="Returns",
        yaxis_title="Frequency",
        height=400,
        showlegend=False
    )

    return fig

def plot_pvalues_summary(df):
    """Genera gráfica de barras con p-values del test ADF para todas las criptomonedas"""

    cryptos = df['Symbol'].unique()
    results = []

    for crypto in cryptos:
        df_crypto = df[df['Symbol'] == crypto].copy()
        df_crypto['Returns'] = calculate_returns(df_crypto['Close'])

        # ADF tests
        adf_prices = adf_test(df_crypto['Close'])
        adf_returns = adf_test(df_crypto['Returns'].dropna())

        results.append({
            'Crypto': crypto,
            'Prices p-value': adf_prices['p-value'],
            'Returns p-value': adf_returns['p-value'],
            'Prices Stationary': adf_prices['Is Stationary'],
            'Returns Stationary': adf_returns['Is Stationary']
        })

    df_results = pd.DataFrame(results)

    # Crear gráfica de barras agrupadas
    fig = go.Figure()

    # Barras para Prices
    fig.add_trace(go.Bar(
        name='Prices p-value',
        x=df_results['Crypto'],
        y=df_results['Prices p-value'],
        marker_color='#3498db',
        text=df_results['Prices p-value'].round(4),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Prices p-value: %{y:.4f}<br>Status: %{customdata}<extra></extra>',
        customdata=['Stationary' if x else 'Non-Stationary (Random Walk)' for x in df_results['Prices Stationary']]
    ))

    # Barras para Returns
    fig.add_trace(go.Bar(
        name='Returns p-value',
        x=df_results['Crypto'],
        y=df_results['Returns p-value'],
        marker_color='#e74c3c',
        text=df_results['Returns p-value'].round(4),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Returns p-value: %{y:.4f}<br>Status: %{customdata}<extra></extra>',
        customdata=['Stationary' if x else 'Non-Stationary' for x in df_results['Returns Stationary']]
    ))

    # Línea de significancia (0.05)
    fig.add_hline(y=0.05, line_dash="dash", line_color="red", line_width=2,
                  annotation_text="Significance Level (α=0.05)",
                  annotation_position="right")

    fig.update_layout(
        title='<b>Random Walk Test Summary: ADF p-values Comparison</b><br><sub>Values > 0.05 indicate Non-Stationary (Random Walk)</sub>',
        xaxis_title='Cryptocurrency',
        yaxis_title='p-value',
        barmode='group',
        height=500,
        plot_bgcolor='white',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=True, gridcolor='lightgray'),
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)'),
        hovermode='x'
    )

    return fig, df_results

# ============================================================================
# GENERACIÓN DE REPORTE HTML
# ============================================================================

def generate_summary_table(df_results):
    """Genera tabla HTML con resumen de resultados ADF"""

    html = """
    <table class="table table-striped table-hover">
        <thead class="table-dark">
            <tr>
                <th>Cryptocurrency</th>
                <th>Prices p-value</th>
                <th>Prices Status</th>
                <th>Returns p-value</th>
                <th>Returns Status</th>
            </tr>
        </thead>
        <tbody>
    """

    for _, row in df_results.iterrows():
        prices_status = '<span class="badge bg-success">Stationary</span>' if row['Prices Stationary'] else '<span class="badge bg-danger">Random Walk</span>'
        returns_status = '<span class="badge bg-success">Stationary</span>' if row['Returns Stationary'] else '<span class="badge bg-danger">Non-Stationary</span>'

        html += f"""
            <tr>
                <td><strong>{row['Crypto']}</strong></td>
                <td>{row['Prices p-value']:.4f}</td>
                <td>{prices_status}</td>
                <td>{row['Returns p-value']:.4f}</td>
                <td>{returns_status}</td>
            </tr>
        """

    html += """
        </tbody>
    </table>
    """

    return html

def generate_statistics_table(df_crypto, crypto_name):
    """Genera tabla de estadísticas descriptivas"""

    stats = {
        'Metric': ['Mean Return', 'Std Dev (Volatility)', 'Skewness', 'Kurtosis',
                   'Min Return', 'Max Return', 'Sharpe Ratio (approx)'],
        'Value': [
            f"{df_crypto['Returns'].mean():.6f}",
            f"{df_crypto['Returns'].std():.6f}",
            f"{df_crypto['Returns'].skew():.4f}",
            f"{df_crypto['Returns'].kurtosis():.4f}",
            f"{df_crypto['Returns'].min():.6f}",
            f"{df_crypto['Returns'].max():.6f}",
            f"{df_crypto['Returns'].mean() / df_crypto['Returns'].std() * np.sqrt(252):.4f}"
        ]
    }

    df_stats = pd.DataFrame(stats)
    return df_stats.to_html(index=False, classes='table table-striped')

def generate_adf_table(adf_results_prices, adf_results_returns):
    """Genera tabla de resultados ADF test"""

    table_html = f"""
    <table class="table table-bordered">
        <thead>
            <tr>
                <th>Test</th>
                <th>Series</th>
                <th>ADF Statistic</th>
                <th>p-value</th>
                <th>Conclusion</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Prices</td>
                <td>Close Price</td>
                <td>{adf_results_prices['Test Statistic']:.4f}</td>
                <td>{adf_results_prices['p-value']:.4f}</td>
                <td>{'<span style="color:green">Stationary</span>' if adf_results_prices['Is Stationary'] else '<span style="color:red">Non-Stationary (Random Walk)</span>'}</td>
            </tr>
            <tr>
                <td>Returns</td>
                <td>Daily Returns</td>
                <td>{adf_results_returns['Test Statistic']:.4f}</td>
                <td>{adf_results_returns['p-value']:.4f}</td>
                <td>{'<span style="color:green">Stationary</span>' if adf_results_returns['Is Stationary'] else '<span style="color:red">Non-Stationary</span>'}</td>
            </tr>
        </tbody>
    </table>
    """
    return table_html

def generate_acf_interpretation_box(crypto_name):
    """Genera caja de interpretación para gráficas ACF"""
    return f"""
    <div class="alert alert-success" style="background-color: #e8f5e9; border-left: 4px solid #4caf50;">
        <h5>📚 How to Interpret ACF Plots for {crypto_name}</h5>
        <ul>
            <li><strong>ACF Prices (High & Slow Decay):</strong> Strong evidence of Random Walk → Prices are unpredictable</li>
            <li><strong>ACF Returns (≈ 0):</strong> Returns are independent → Cannot predict future from past returns</li>
            <li><strong>ACF Returns² (High values):</strong> Strong volatility clustering → Risk comes in waves</li>
        </ul>
        <p><strong>Investment Implication:</strong> You cannot predict price direction, but you CAN anticipate volatile periods!</p>
    </div>
    """

def create_html_report(df, output_file='crypto_analysis_report.html'):
    """Genera reporte HTML completo con ACF plots y resumen de p-values"""

    # Generar resumen de p-values primero
    fig_pvalues, df_results = plot_pvalues_summary(df)

    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Cryptocurrency Time Series Analysis Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
        <style>
            body { font-family: 'Arial', sans-serif; background-color: #f5f5f5; }
            .container { max-width: 1600px; margin: 20px auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
            h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }
            h2 { color: #34495e; margin-top: 40px; margin-bottom: 20px; }
            h3 { color: #7f8c8d; margin-top: 30px; }
            .crypto-section { margin-bottom: 60px; padding: 20px; background: #fafafa; border-radius: 8px; }
            .summary-section { margin-bottom: 40px; padding: 30px; background: #f8f9fa; border-radius: 8px; border-left: 5px solid #3498db; }
            .table { margin: 20px 0; }
            .alert-info { background-color: #e3f2fd; border-left: 4px solid #2196f3; }
            .alert-success { background-color: #e8f5e9; border-left: 4px solid #4caf50; }
            .alert-warning { background-color: #fff3e0; border-left: 4px solid #ff9800; }
            .plotly-graph-div { margin: 20px 0; }
            .badge { padding: 5px 10px; }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>📊 Cryptocurrency Time Series Analysis Report</h1>
            <p class="lead">Complete analysis of cryptocurrency price behavior and predictability</p>
            <p><strong>Report Generated:</strong> """ + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + """</p>

            <div class="alert alert-info">
                <h4>📌 Executive Summary</h4>
                <p>This report analyzes cryptocurrency price behavior through time series analysis,
                focusing on random walk properties, return characteristics, and volatility patterns.
                <strong>ACF plots</strong> provide visual evidence of predictability (or lack thereof).</p>
            </div>

            <div class="summary-section">
                <h2>🎯 Random Walk Test - Summary Results</h2>
                <p class="lead">Overview of stationarity tests for all cryptocurrencies</p>

                <div class="alert alert-warning">
                    <h5>📖 How to Read This Summary:</h5>
                    <ul>
                        <li><strong>p-value > 0.05:</strong> Non-Stationary (Random Walk) → <span class="badge bg-danger">Not Predictable</span></li>
                        <li><strong>p-value < 0.05:</strong> Stationary → <span class="badge bg-success">May Have Patterns</span></li>
                        <li><strong>Expected for Cryptos:</strong> Prices should be Random Walk, Returns should be Stationary</li>
                    </ul>
                </div>

                <h3>Visual Comparison of p-values</h3>
                <div id="plot_pvalues_summary"></div>

                <h3>Detailed Results Table</h3>
                """ + generate_summary_table(df_results) + """

                <div class="alert alert-info mt-4">
                    <h5>💡 Key Insights:</h5>
                    <p><strong>What this means for investors:</strong> If prices show high p-values (Random Walk),
                    technical analysis and price predictions are unreliable. Focus on fundamentals and long-term strategies instead.</p>
                </div>
            </div>
    """

    # Obtener criptomonedas únicas
    cryptos = df['Symbol'].unique()

    for crypto in cryptos:
        print(f"Processing {crypto}...")
        df_crypto = df[df['Symbol'] == crypto].copy()
        df_crypto['Returns'] = calculate_returns(df_crypto['Close'])

        # Tests estadísticos
        adf_prices = adf_test(df_crypto['Close'], f"{crypto} Prices")
        adf_returns = adf_test(df_crypto['Returns'].dropna(), f"{crypto} Returns")

        html_content += f"""
        <div class="crypto-section">
            <h2>🪙 {crypto}</h2>

            <h3>1. Statistical Summary</h3>
            {generate_statistics_table(df_crypto, crypto)}

            <h3>2. Random Walk Test (Augmented Dickey-Fuller)</h3>
            {generate_adf_table(adf_prices, adf_returns)}

            <h3>3. Price and Returns Evolution</h3>
            <div id="plot_price_returns_{crypto}"></div>

            <h3>4. ACF Analysis</h3>
            {generate_acf_interpretation_box(crypto)}

            <div id="plot_acf_prices_{crypto}"></div>
            <div id="plot_acf_returns_{crypto}"></div>
            <div id="plot_acf_returns2_{crypto}"></div>

            <h3>5. Volatility Clustering</h3>
            <div id="plot_volatility_{crypto}"></div>

            <h3>6. Returns Distribution</h3>
            <div id="plot_distribution_{crypto}"></div>
        </div>
        """

        # Generar gráficas
        fig_price = plot_price_and_returns(df_crypto, crypto)
        acf_plots = plot_acf_analysis_complete(df_crypto, crypto)
        fig_vol = plot_volatility_clustering(df_crypto, crypto)
        fig_dist = plot_returns_distribution(df_crypto, crypto)

        # Insertar scripts de Plotly
        html_content += f"""
        <script>
            // Price and Returns
            var plot1 = {fig_price.to_json()};
            Plotly.newPlot('plot_price_returns_{crypto}', plot1.data, plot1.layout);

            // ACF Plots
            var acf_prices = {acf_plots[0].to_json()};
            Plotly.newPlot('plot_acf_prices_{crypto}', acf_prices.data, acf_prices.layout);

            var acf_returns = {acf_plots[1].to_json()};
            Plotly.newPlot('plot_acf_returns_{crypto}', acf_returns.data, acf_returns.layout);

            var acf_returns2 = {acf_plots[2].to_json()};
            Plotly.newPlot('plot_acf_returns2_{crypto}', acf_returns2.data, acf_returns2.layout);

            // Volatility
            var plot_vol = {fig_vol.to_json()};
            Plotly.newPlot('plot_volatility_{crypto}', plot_vol.data, plot_vol.layout);

            // Distribution
            var plot_dist = {fig_dist.to_json()};
            Plotly.newPlot('plot_distribution_{crypto}', plot_dist.data, plot_dist.layout);
        </script>
        """

    # Añadir script para gráfica de resumen de p-values
    html_content += f"""
        <script>
            // P-values Summary
            var plot_pvalues = {fig_pvalues.to_json()};
            Plotly.newPlot('plot_pvalues_summary', plot_pvalues.data, plot_pvalues.layout);
        </script>
    """

    html_content += """
        </div>
    </body>
    </html>
    """

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"\n✅ Report generated successfully: {output_file}")
    return df_results

# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def main():
    """Función principal para ejecutar el análisis completo"""

    # Cargar datos
    print("Loading cryptocurrency data...")
    df = load_data('/content/crypto_prices_2020_2025_daily.csv')  # Ajusta el nombre del archivo

    print(f"Data loaded: {len(df)} rows")
    print(f"Cryptocurrencies: {df['Symbol'].unique()}")
    print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

    # Generar reporte HTML
    print("\nGenerating HTML report with ACF plots and p-values summary...")
    df_results = create_html_report(df, 'crypto_analysis_report.html')

    print("\n" + "="*60)
    print("✅ ANALYSIS COMPLETE!")
    print("="*60)
    print("\n📄 Report: 'crypto_analysis_report.html'")
    print("\n📊 Summary of Random Walk Tests:")
    print("-" * 60)
    for _, row in df_results.iterrows():
        print(f"\n{row['Crypto']}:")
        print(f"  Prices:  p-value = {row['Prices p-value']:.4f} → {'Random Walk ❌' if not row['Prices Stationary'] else 'Stationary ✅'}")
        print(f"  Returns: p-value = {row['Returns p-value']:.4f} → {'Stationary ✅' if row['Returns Stationary'] else 'Non-Stationary ❌'}")

    print("\n" + "="*60)
    print("💡 Key Features:")
    print("   ✅ Summary section with p-values bar chart")
    print("   ✅ ACF plots (without PACF) - cleaner visualization")
    print("   ✅ No zero lines in ACF plots")
    print("   ✅ Detailed interpretation guides")
    print("="*60)

# Ejecutar análisis
if __name__ == "__main__":
    main()